In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
data=pd.read_excel("D:\\DATASETS\\House_Rent_Dataset new.xlsx")

In [32]:
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [33]:
data.shape

(4746, 12)

In [35]:
data=data.drop(['Posted On','Floor','Area Locality','Tenant Preferred','Point of Contact'],axis=1)

In [36]:
data.head()

,BHK,Rent,Size,Area Type,City,Furnishing Status,Bathroom
0,2,10000,1100,Super Area,Kolkata,Unfurnished,2
1,2,20000,800,Super Area,Kolkata,Semi-Furnished,1
2,2,17000,1000,Super Area,Kolkata,Semi-Furnished,1
3,2,10000,800,Super Area,Kolkata,Unfurnished,1
4,2,7500,850,Carpet Area,Kolkata,Unfurnished,1


In [37]:
data['BHK'].unique()

array([2, 1, 3, 6, 4, 5], dtype=int64)

In [38]:
data.isnull().sum()

BHK                  0
Rent                 0
Size                 0
Area Type            0
City                 0
Furnishing Status    0
Bathroom             0
dtype: int64

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4746 non-null   int64 
 1   Rent               4746 non-null   int64 
 2   Size               4746 non-null   int64 
 3   Area Type          4746 non-null   object
 4   City               4746 non-null   object
 5   Furnishing Status  4746 non-null   object
 6   Bathroom           4746 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 259.7+ KB


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [41]:
X_train,X_test,y_train,y_test=train_test_split(data.drop('Area Type',axis=1),data['Area Type'],random_state=56)

In [63]:
X_train

,BHK,Rent,Size,City,Furnishing Status,Bathroom
3342,2,12000,750,Chennai,Unfurnished,2
6,2,10000,700,Kolkata,Unfurnished,2
1559,1,6000,400,Bangalore,Semi-Furnished,1
2372,2,14000,800,Bangalore,Semi-Furnished,1
4549,1,7000,300,Hyderabad,Unfurnished,1
...,...,...,...,...,...,...
1146,4,250000,2500,Mumbai,Furnished,4
1259,1,28500,565,Mumbai,Furnished,2
3264,3,20000,1650,Chennai,Semi-Furnished,3
399,2,8000,650,Kolkata,Unfurnished,1


In [42]:
num_col=[i for i in X_train if X_train[i].dtype !='object']
cat_col=[i for i in X_train if X_train[i].dtype =='object']

In [43]:
print(num_col)
print(cat_col)

['BHK', 'Rent', 'Size', 'Bathroom']
['City', 'Furnishing Status']


In [44]:
cat_pipe=Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

In [45]:
num_pipe=Pipeline(steps=[
    ('scaler',StandardScaler())
])

In [46]:
preprocessor=ColumnTransformer(transformers=[
    ('num',num_pipe,['Rent','Size']),
    ('cat',cat_pipe,cat_col)
])

In [ ]:
model=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',DecisionTreeClassifier(criterion='gini',splitter='random',max_depth=5))
])

In [60]:
model.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Rent', 'Size']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City',
                                                   'Furnishing Status'])])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=5, splitter='random'))])

In [61]:
y_pred=model.predict(X_test)

In [62]:
print("Accuracy:",accuracy_score(y_test,y_pred))

Accuracy: 0.6680707666385847


In [71]:
confusion_matrix(y_test,y_pred)

array([[  0,   0,   1],
       [  0, 366, 216],
       [  0, 177, 427]], dtype=int64)

In [69]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

  Built Area       0.00      0.00      0.00         1
 Carpet Area       0.67      0.63      0.65       582
  Super Area       0.66      0.71      0.68       604

    accuracy                           0.67      1187
   macro avg       0.45      0.45      0.44      1187
weighted avg       0.67      0.67      0.67      1187



c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [65]:
X_train.columns

Index(['BHK', 'Rent', 'Size', 'City', 'Furnishing Status', 'Bathroom'], dtype='object')

In [66]:
input_data=[[3,12500	,800,'Chennai','furnished',3]]
col_names=['BHK', 'Rent', 'Size', 'City', 'Furnishing Status', 'Bathroom']
input_df=pd.DataFrame(input_data,columns=col_names)

In [67]:
model.predict(input_df)

array(['Super Area'], dtype=object)